# プログラムでモデルを作る

## 異世界で動く振り子

In [ ]:
# 15・1・1
# 50センチ刻みで2メールまでふりこの長さを変えたときの周期
periods = [0, 2.0, 3.0, 3.5, 4.0]

## モデルを検証する

In [ ]:
# 15・1・2
import numpy as np
import matplotlib.pyplot as plt

# データ(説明変数)
lengths = np.array([0, 0.5, 1.0, 1.5, 2.0])
# データ(従属変数)
periods = np.array([0, 2.0, 3.0, 3.5, 4.0])

# 傾き候補
slopes = [1.7, 2.0, 2.3, 2.6, 2.9]

# グラフ描画
plt.scatter(lengths, periods, color="blue")
for m in slopes:
    plt.plot(lengths, m * lengths, label=f"slope={m}")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 15・1・3
from sklearn.metrics import r2_score  # 決定係数を計算する関数

r2_results = {}
for m in slopes:
    y_pred = m * lengths
    r2 = r2_score(periods, y_pred)
    r2_results[m] = r2

print("各傾きの決定係数:")
for m, r2 in r2_results.items():
    print(f"slope={m}: R^2={r2:.3f}")

In [ ]:
# 15・1・4
from sklearn.linear_model import LinearRegression

# データ
lengths = np.array([0, 0.5, 1.0, 1.5, 2.0])
periods = np.array([0, 2.0, 3.0, 3.5, 4.0])

# 線形回帰（切片を0に固定）
X = lengths.reshape(-1, 1)
model = LinearRegression(fit_intercept=False)
model.fit(X, periods)

# 最適傾き
m_opt = model.coef_[0]

# 予測と R^2
y_pred = model.predict(X)
r2 = r2_score(periods, y_pred)

print(f"最適傾き m = {m_opt}")
print(f"決定係数 R^2 = {r2:.3f}")

## パラメータとしての累乗数

In [ ]:
# 15・1・5
from sklearn.preprocessing import PolynomialFeatures

# データ
lengths = np.array([0, 0.5, 1.0, 1.5, 2.0])
periods = np.array([0, 2.0, 3.0, 3.5, 4.0])

L_plot = np.linspace(0, 2.0, 100)

plt.scatter(lengths, periods, color="blue")

# 0.5次（平方根近似）
model_sqrt = LinearRegression(fit_intercept=False)
model_sqrt.fit(np.sqrt(lengths).reshape(-1, 1), periods)
plt.plot(L_plot, model_sqrt.predict(np.sqrt(L_plot).reshape(-1, 1)), label="0.5")
print(model_sqrt.coef_[0])

# 1〜3次
for deg in [1, 2, 3]:
    poly = PolynomialFeatures(degree=deg, include_bias=False)
    X_poly = poly.fit_transform(lengths.reshape(-1, 1))
    model_poly = LinearRegression(fit_intercept=False)
    model_poly.fit(X_poly, periods)
    plt.plot(L_plot, model_poly.predict(poly.transform(L_plot.reshape(-1, 1))), label=f"{deg}")

plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 15・1・6
from sklearn.metrics import r2_score

r2_scores = {}

# 0.5次
y_pred_sqrt = model_sqrt.predict(np.sqrt(lengths).reshape(-1, 1))
r2_scores[0.5] = r2_score(periods, y_pred_sqrt)

# 1〜3次
for deg in [1, 2, 3]:
    poly = PolynomialFeatures(degree=deg, include_bias=False)
    X_poly = poly.fit_transform(lengths.reshape(-1, 1))
    model_poly = LinearRegression(fit_intercept=False)
    model_poly.fit(X_poly, periods)
    y_pred_poly = model_poly.predict(X_poly)
    r2_scores[deg] = r2_score(periods, y_pred_poly)

print(r2_scores)

In [ ]:
# 15・1・7
m = model_sqrt.coef_[0]

# g を推定
g_est = (2 * np.pi / m) ** 2

print(f"0.5次モデルの傾き m = {m:.3f}")
print(f"推定重力加速度 g = {g_est:.3f}m/s^2")